In [ ]:
import pandas as pd
import numpy as np
import arviz as az

df = pd.read_csv("Prices.csv")

y  = df["Price"].values
x1 = df["Speed"].values                 
x2 = np.log(df["HardDrive"].values)     

In [ ]:
import pymc as pm

# a) (1p) Using weakly informative prior distributions for the parameters α, β1, β2 and σ, use PyMC to simulate a
# sufficiently large sample from the posterior distribution.

with pm.Model() as model:
    alpha = pm.Normal("alpha", mu=0, sigma=1000)
    beta1 = pm.Normal("beta1", mu=0, sigma=100)
    beta2 = pm.Normal("beta2", mu=0, sigma=100)
    sigma = pm.HalfNormal("sigma", sigma=1000)

    
    mu = alpha + beta1 * x1 + beta2 * x2

    
    price = pm.Normal("price", mu=mu, sigma=sigma, observed=y)

    
    idata = pm.sample(
        draws=2000,       
        tune=2000,       
        target_accept=0.9,
        chains=4,
        cores=4,
        return_inferencedata=True
    )


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta1, beta2, sigma]


Output()

Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 6 seconds.


In [28]:
# import arviz as az


# az.summary(idata, var_names=["alpha", "beta1", "beta2", "sigma"])
# az.plot_trace(idata, var_names=["alpha", "beta1", "beta2", "sigma"])


In [ ]:
# b) (0.5p) Obtain 95% HDI estimates for the parameters β1 and β2.

# c) (0.5p) Based on the results, are processor frequency and hard disk size useful predictors of the sale price?

az.summary(idata, var_names=["beta1", "beta2"], hdi_prob=0.95)

# putem vedea ca beta1 si beta2 sunt esmnificative pentru ca
# sunt intervale destul de stramte (precise) si ofera informatii despre cat ar creste pretul

# beta1-> pt 1 unitate de putere de procesor pretul += [2.36$, 7.08$] - destul de putin relativ la pretul unui laptop
# beta1-> pt 1 unitate de log size pretul += [236$, 375$]

# in plus, ele nu trec prin 0 si sunt mereu pozitive, deci pretul este proportional cu aceste atribute



,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta1,4.667,1.209,2.221,6.981,0.018,0.016,4750.0,3865.0,1.0
beta2,307.961,35.881,237.349,375.842,0.640,0.476,3147.0,3786.0,1.0


In [ ]:
# d) (0.5p) Now suppose a consumer is interested in a computer with a frequency of 33 MHz and a hard disk of 540
# MB. Simulate draws from the expected sale price (μ) and construct a 90% HDI interval for this price.

x1_new = 33
x2_new = np.log(540)

alpha_s = idata.posterior["alpha"].values.reshape(-1)
beta1_s = idata.posterior["beta1"].values.reshape(-1)
beta2_s = idata.posterior["beta2"].values.reshape(-1)

mu_samples = alpha_s + beta1_s * x1_new + beta2_s * x2_new

hdi_90 = az.hdi(mu_samples, hdi_prob=0.90)
mu_mean = mu_samples.mean()

print("Posterior mean of μ:", mu_mean)
print("90% HDI for μ:", hdi_90)


Posterior mean of μ: 2286.597159878468
90% HDI for μ: [2224.59014996 2351.42586792]


In [31]:
# e) (1p) Instead, suppose this consumer wants to predict the sale price of a computer with this frequency and hard
# disk size. Simulate draws from the posterior predictive distribution and use these simulated draws to find a 90%
# HDI prediction interval.

sigma_s = idata.posterior["sigma"].values.reshape(-1)
y_pred_s = np.random.normal(loc=mu_samples, scale=sigma_s)


pred_hdi_90 = az.hdi(y_pred_s, hdi_prob=0.90)
pred_mean = y_pred_s.mean()

print("Posterior predictive mean price:", pred_mean)
print("90% HDI prediction interval for price:", pred_hdi_90)

Posterior predictive mean price: 2292.7697385170936
90% HDI prediction interval for price: [1482.51096194 3132.7692058 ]


In [32]:
# (0.5p) Does the fact that the manufacturer is premium affect the price in any way? Justify.

x3 = df["Premium"].values

x3 = [1 if val == "yes" else 0 for val in x3]

with pm.Model() as model:
    alpha = pm.Normal("alpha", mu=0, sigma=1000)
    beta1 = pm.Normal("beta1", mu=0, sigma=100)
    beta2 = pm.Normal("beta2", mu=0, sigma=100)
    sigma = pm.HalfNormal("sigma", sigma=1000)

    beta3 = pm.Normal("beta3", mu=0, sigma=100) # pt premium

    
    mu = alpha + beta1 * x1 + beta2 * x2 + beta3 * x3

    
    price = pm.Normal("price", mu=mu, sigma=sigma, observed=y)

    # Posterior sampling (sufficiently large sample)
    idata = pm.sample(
        draws=2000,       
        tune=2000,       
        target_accept=0.9,
        chains=4,
        cores=4,
        return_inferencedata=True
    )


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta1, beta2, sigma, beta3]


Output()

/Users/stefaniancu/Documents/An 3 Seminare/PMP/PMP-2025/.venv/lib/python3.13/site-packages/pymc/step_methods/hmc/quadpotential.py:316: RuntimeWarning: overflow encountered in dot
  return 0.5 * np.dot(x, v_out)


Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 7 seconds.


In [33]:

az.summary(idata, var_names=["beta3"], hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta3,-189.506,60.399,-302.706,-70.157,0.761,0.68,6306.0,5563.0,1.0


In [ ]:

mean_premium = df[df["Premium"] == "yes"]["Price"].mean()
mean_nonpremium = df[df["Premium"] == "no"]["Price"].mean()

print("Mean price for premium laptops:", mean_premium)
print("Mean price for non-premium laptops:", mean_nonpremium)

# deci putem vedea ca atributul premium aduce o informatie despre pret
# si spre uimirea mea, aparent in acest dataset laptopurile premium sunt mai ieftine decat cele non-premium

# de asta am facut si media, sa vad ca intervalul hdi nu este cumva gresit si ca sa ma verific cu
# intervalul calculat la punctul e)

Mean price for premium laptops: 2235.708609271523
Mean price for non-premium laptops: 2367.276595744681
